BASE SALÁRIO CONTRIBUIÇÃO
        BIBLIOTECAS

In [1]:
import pandas as pd
import pyperclip
import os
import numpy as np

VARIÁVEIS

In [2]:
# 1200x1010

DATA_INICIO = '2018-01-01 00:00:00'
DATA_FIM = '2025-12-31 23:59:59'
PALAVRAS_CHAVE = ["Vlr", "Valor", "vlr", "valor", "Vr", "vr", 
                  "base", "Base", "Salário", "Salario", "salário", 
                  "salario", "Provento", "provento", "Desconto", "desconto", 
                  "diferenca", "Diferenca", "diferença", "Diferença"]

CAMINHOS

In [3]:
a = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\s1200')
b = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\s1010')
c = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\s2299')
d = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\s5011_base_cp')

print(len(a))
print(len(b))
print(len(c))
print(len(d))


363350
539
15351
1448


CONSOLIDADO 1200x1010

In [4]:
# 1200x1010

relatorios = ['1200', '1010']

sys_df_1010 = None
sys_df_1200 = None
df_sys = None
parquet_path_sys = None

for relatorio in relatorios:
    relatorio_sis = f's{relatorio}'

    try:
        parquet_path_sys = f'C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\{relatorio_sis}\\'

        df_sys = pd.read_parquet(parquet_path_sys, engine='pyarrow')

        df_sys['periodo'] = (
                df_sys['id'].astype(str).str[17:21] + '-' +
                df_sys['id'].astype(str).str[21:23] + '-' +
                df_sys['id'].astype(str).str[23:25] + ' ' +
                df_sys['id'].astype(str).str[25:27] + ':' +
                df_sys['id'].astype(str).str[27:29]
        )

        df_sys['periodo'] = pd.to_datetime(df_sys['periodo'])
        dt_ini = pd.to_datetime(DATA_INICIO)
        dt_fim = pd.to_datetime(DATA_FIM)

        if relatorio != '1010':
            df_sys = df_sys[(df_sys['periodo'] >= dt_ini) & (df_sys['periodo'] <= dt_fim)]

    except Exception as e:
        print(f"Erro ao carregar {parquet_path_sys}: {e}")

    if relatorio == '1010':
        sys_df_1010 = df_sys
    elif relatorio == '1200':
        sys_df_1200 = df_sys

# Renomeia colunas
s1200 = sys_df_1200.copy().reset_index()
s1010 = sys_df_1010.copy()


In [5]:
# 1200x1010

s1200.rename(columns={
            'codigo_rubrica': 'cod_rubrica',
            'identificador_rubrica': 'id_rubrica',
            'periodo': 'periodo',
            'id': 'id'
        }, inplace=True)

s1010.rename(columns={
            'codigo_rubrica': 'cod_rubrica',
            'identificacao_rubrica': 'id_rubrica',
            'periodo': 'periodo_1010',
            'id': 'id_1010'
        }, inplace=True)

s1200['periodo'] = pd.to_datetime(s1200['periodo'])
s1010['periodo_1010'] = pd.to_datetime(s1010['periodo_1010'])

# 1ª Consolidação: período_1010 <= período
merged = s1200.merge(
            s1010,
            on=['cod_rubrica', 'id_rubrica'],
            suffixes=('', '_1010')
        )

valid = merged[merged['periodo_1010'] <= merged['periodo']]
idx = valid.groupby(['id', 'cod_rubrica', 'id_rubrica', 'index'])['periodo_1010'].idxmax()
s12_final = valid.loc[idx].copy()

colunas_base = sys_df_1200.columns.drop('periodo')
colunas_extras = ['index', 'id_1010', 'descricao_rubrica', 'tipo_da_rubrica', 'incidencia_previdencia_social']
s12_final.rename(columns={
    'numero_do_recibo': 'numero_do_recibo',
    'periodo': 'periodo',
    'id_rubrica': 'identificador_rubrica',
    'cod_rubrica': 'codigo_rubrica',
    'id': 'id'
}, inplace=True)
s12_final = s12_final[list(colunas_base) + colunas_extras]

# Filtra não consolidados
s1200_restante = s1200[~s1200['index'].isin(s12_final['index'])]

merged_restante = s1200_restante.merge(
            s1010,
            on=['cod_rubrica', 'id_rubrica'],
            suffixes=('', '_1010'),
            how='left'
        )

merged_restante.rename(columns={
            'numero_do_recibo': 'numero_do_recibo',
            'periodo': 'periodo',
            'id_rubrica': 'identificador_rubrica',
            'cod_rubrica': 'codigo_rubrica',
            'id': 'id'
        }, inplace=True)

merged_restante = merged_restante[[col for col in s12_final.columns]]

# Consolidação final
df_final = pd.concat([s12_final, merged_restante], ignore_index=True)

s1010_utilizados = df_final[['id_1010']].dropna().drop_duplicates()

s1010_usado_completo = s1010[s1010['id_1010'].isin(s1010_utilizados['id_1010'])].copy()

s1010_usado_completo['periodo_1010'] = pd.to_datetime(
    s1010_usado_completo['periodo_1010'], errors='coerce'
).astype('datetime64[ns]')

df_final_1200 = df_final.drop_duplicates(subset=['index', 'id', 'codigo_rubrica', 'identificador_rubrica'])\
    .drop(columns=['index', 'id_1010'], errors='ignore').fillna('')

df_periods_s1200_s1010 = df_final_1200.drop_duplicates(subset=['periodo_de_apuracao'])

CONSOLIDADO 2299x1010

In [7]:
# 2299x1010

relatorios = ['2299', '1010']

sys_df_1010 = None
sys_df_2299 = None
df_sys = None
parquet_path_sys = None

for relatorio in relatorios:
    relatorio_sis = f's{relatorio}'

    try:
        parquet_path_sys = f'C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\{relatorio_sis}\\'

        df_sys = pd.read_parquet(parquet_path_sys, engine='pyarrow')

        df_sys['periodo'] = (
                df_sys['id'].astype(str).str[17:21] + '-' +
                df_sys['id'].astype(str).str[21:23] + '-' +
                df_sys['id'].astype(str).str[23:25] + ' ' +
                df_sys['id'].astype(str).str[25:27] + ':' +
                df_sys['id'].astype(str).str[27:29]
        )

        df_sys['periodo'] = pd.to_datetime(df_sys['periodo'])
        dt_ini = pd.to_datetime(DATA_INICIO)
        dt_fim = pd.to_datetime(DATA_FIM)

        if relatorio != '1010':
            df_sys = df_sys[(df_sys['periodo'] >= dt_ini) & (df_sys['periodo'] <= dt_fim)]

    except Exception as e:
        print(f"Erro ao carregar {parquet_path_sys}: {e}")

    if relatorio == '1010':
        sys_df_1010 = df_sys
    elif relatorio == '2299':
        sys_df_2299 = df_sys

# Renomeia colunas
s2299 = sys_df_2299.copy().reset_index()
s1010 = sys_df_1010.copy()

In [8]:
# 2299x1010

s2299.rename(columns={
            'codigo_rubrica': 'cod_rubrica',
            'identificador_tabela_rubricas': 'id_rubrica',
            'periodo': 'periodo_2299',
            'id': 'id_s2299'
        }, inplace=True)

s1010.rename(columns={
            'codigo_rubrica': 'cod_rubrica',
            'identificacao_rubrica': 'id_rubrica',
            'periodo': 'periodo_1010',
            'id': 'id_1010'
        }, inplace=True)

s2299['periodo_2299'] = pd.to_datetime(s2299['periodo_2299'])
s1010['periodo_1010'] = pd.to_datetime(s1010['periodo_1010'])

# 1ª Consolidação: período_1010 <= período
merged = s2299.merge(
            s1010,
            on=['cod_rubrica', 'id_rubrica'],
            suffixes=('_s2299', '_1010')
        )

# Filtrar por datas válidas
valid = merged[merged['periodo_1010'] <= merged['periodo_2299']]

# Agrupar por ID completo + rubrica, e pegar o idxmax
idx = valid.groupby(['id_s2299', 'cod_rubrica', 'id_rubrica', 'index'])['periodo_1010'].idxmax()

# Selecionar as linhas finais
s2299_final = valid.loc[idx].copy()

colunas_s2299 = [col for col in sys_df_2299.columns]

s2299_final.rename(columns={
            'id_s2299': 'id',
            'periodo_2299': 'periodo',
            'id_rubrica': 'identificador_tabela_rubricas',
            'cod_rubrica': 'codigo_rubrica',
            'numero_do_recibo_s2299': 'numero_do_recibo',
            'identificacao_ambiente_s2299': 'identificacao_ambiente',
            'processo_emissao_s2299': 'processo_emissao',
            'versao_aplicativo_s2299': 'versao_aplicativo',
            'tipo_inscricao_empregador_s2299': 'tipo_inscricao_empregador',
            'numero_inscricao_empregador_s2299': 'numero_inscricao_empregador',
            'observacao_s2299': 'observacao'
        }, inplace=True)

# Juntar com as 3 colunas desejadas do s1010
colunas_s2299 = [col for col in colunas_s2299 if col not in ['id']]

colunas_final = ['id', 'id_1010', 'index'] + colunas_s2299 + [
            'descricao_rubrica', 'tipo_da_rubrica', 'incidencia_previdencia_social'
        ]

colunas_final = [col for col in colunas_final if col not in ['periodo']]

s2299_final = s2299_final[colunas_final]

s2299_2_ = s2299[~s2299['index'].isin(s2299_final['index'])]

merged = s2299_2_.merge(
            s1010,
            on=['cod_rubrica', 'id_rubrica'],
            suffixes=('', '_1010'),
            how='left'
        )

merged_ = merged.rename(columns={
            'periodo_2299': 'periodo',
            'id_s2299': 'id',
            'cod_rubrica': 'codigo_rubrica',
            'id_rubrica': 'identificador_tabela_rubricas'
        }).drop_duplicates(subset=['index', 'id', 'codigo_rubrica', 'identificador_tabela_rubricas'])

colunas_final = [col for col in colunas_final if col not in ['periodo']]

merged_selected = merged_[colunas_final]

merged_com_s2299_final = pd.concat([s2299_final, merged_selected])

df_final_2299 = merged_com_s2299_final.drop_duplicates(subset=['index', 'id', 'codigo_rubrica', 'identificador_tabela_rubricas'])\
    .drop(columns=['index', 'id_1010'], errors='ignore').fillna('')

s5011 BASE CONTRIBUIÇÃO PREVIDENCIÁRIA

In [9]:
# Leitura do parquet
df = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\s5011_base_cp')

# Filtro pelo ano da coluna 'periodo_de_apuracao'
ano_ini = dt_ini.year
ano_fim = dt_fim.year

temp_s5011_base_cp = df[
    df['periodo_de_apuracao'].str[:4].astype(int).between(ano_ini, ano_fim)
].copy()

INICIO

In [10]:
s1200_filtered = df_final_1200[
    (df_final_1200['tipo_da_rubrica'] == '1 - Vencimento, provento ou pensão') &
    (df_final_1200['incidencia_previdencia_social'].isin(['12 - 13º salário', '11 - Mensal']))
].copy()

s1200_filtered['valor_total'] = s1200_filtered['valor_total'].replace(',', '.').astype(float)

cte_provento_1200 = (
    s1200_filtered
    .groupby('periodo_de_apuracao')['valor_total']
    .sum()
    .reset_index()
    .rename(columns={'valor_total': 'provento_1200'})
)


In [11]:
s2299_filtered = df_final_2299[
    (df_final_2299['tipo_da_rubrica'] == '1 - Vencimento, provento ou pensão') &
    (df_final_2299['incidencia_previdencia_social'].isin(['12 - 13º salário', '11 - Mensal']))
].copy()

s2299_filtered['periodo_de_apuracao'] = s2299_filtered['data_desligamento'].str.slice(0, 7)
s2299_filtered['valor_total_rubrica'] = s2299_filtered['valor_total_rubrica'].str.replace(',', '.').astype(float)

cte_provento_2299 = (
    s2299_filtered
    .groupby('periodo_de_apuracao')['valor_total_rubrica']
    .sum()
    .reset_index()
    .rename(columns={'valor_total_rubrica': 'provento_2299'})
)


In [12]:
s1200_desc = df_final_1200[
    (df_final_1200['tipo_da_rubrica'] == '2 - Desconto') &
    (df_final_1200['incidencia_previdencia_social'].isin(['12 - 13º salário', '11 - Mensal']))
].copy()

s1200_desc['valor_total'] = s1200_desc['valor_total'].replace(',', '.').astype(float)

cte_desconto_1200 = (
    s1200_desc
    .groupby('periodo_de_apuracao')['valor_total']
    .sum()
    .reset_index()
    .rename(columns={'valor_total': 'desconto_1200'})
)


In [13]:
s2299_desc = df_final_2299[
    (df_final_2299['tipo_da_rubrica'] == '2 - Desconto') &
    (df_final_2299['incidencia_previdencia_social'].isin(['12 - 13º salário', '11 - Mensal']))
].copy()

s2299_desc['periodo_de_apuracao'] = s2299_desc['data_desligamento'].str.slice(0, 7)
s2299_desc['valor_total_rubrica'] = s2299_desc['valor_total_rubrica'].str.replace(',', '.').astype(float)

cte_desconto_2299 = (
    s2299_desc
    .groupby('periodo_de_apuracao')['valor_total_rubrica']
    .sum()
    .reset_index()
    .rename(columns={'valor_total_rubrica': 'desconto_2299'})
)


In [14]:
# s5011

cte_base_cpp = (
    temp_s5011_base_cp
    .groupby('periodo_de_apuracao')['vr_bc_cp_00']
    .sum()
    .reset_index()
    .rename(columns={'vr_bc_cp_00': 'base_cpp'})
)


In [15]:
# Base Salário Contribuição

temp_final_report = df_periods_s1200_s1010.copy()

# Fazendo os merges
temp_final_report = temp_final_report.merge(cte_provento_1200, on='periodo_de_apuracao', how='left')
temp_final_report = temp_final_report.merge(cte_provento_2299, on='periodo_de_apuracao', how='left')
temp_final_report = temp_final_report.merge(cte_desconto_1200, on='periodo_de_apuracao', how='left')
temp_final_report = temp_final_report.merge(cte_desconto_2299, on='periodo_de_apuracao', how='left')
temp_final_report = temp_final_report.merge(cte_base_cpp, on='periodo_de_apuracao', how='left')

# Substituindo NaNs por 0
temp_final_report.fillna(0, inplace=True)

# Cálculos finais
temp_final_report['salario_contribuicao'] = round(
    (temp_final_report['provento_1200'] + temp_final_report['provento_2299']) -
    (temp_final_report['desconto_1200'] + temp_final_report['desconto_2299']), 2
)

temp_final_report['total_proventos'] = round(
    temp_final_report['provento_1200'] + temp_final_report['provento_2299'], 2
)

temp_final_report['total_descontos'] = round(
    temp_final_report['desconto_1200'] + temp_final_report['desconto_2299'], 2
)

temp_final_report['diferenca'] = round(
    temp_final_report['base_cpp'] - temp_final_report['salario_contribuicao'], 2
)

# Ordenando
temp_final_report = temp_final_report.sort_values(by='periodo_de_apuracao')


In [17]:
# Base Salário Contribuição

num_linhas, num_colunas = temp_final_report.shape
resultado = f"🔹 Total de Linhas: {num_linhas}\n🔹 Total de Colunas: {num_colunas}\n\n📊 Detalhes das Colunas:\n"

for coluna in temp_final_report.columns:
    linhas_preenchidas = temp_final_report[coluna].count()  # Conta valores não nulos
    
    # Verifica se a coluna está na lista de palavras-chave
    if any(palavra.lower() in coluna.lower() for palavra in PALAVRAS_CHAVE):
            # Soma
            soma_valores = (
                temp_final_report[coluna]
                .replace('', np.nan)
                .astype(float)
                .sum(skipna=True)
            )

            # Exibição em formato brasileiro
            valor_formatado = f"{soma_valores:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

            resultado += f"\n📌 Coluna: {coluna} (🔄 Convertida para Float e Substituição de '.' por ',')\n"
            resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"
            resultado += f"   🔹 Total Somado: {soma_valores:,.2f}\n"
    else:
        resultado += f"\n📌 Coluna: {coluna}\n"
        resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"

# Copiar para a área de transferência
pyperclip.copy(resultado)

# Exibir os resultados
print("\n✅ Resultado copiado para a área de transferência!")


✅ Resultado copiado para a área de transferência!
